In [ ]:
import json
import os
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import datetime
import playlist
reload(playlist)

In [ ]:
playlist_list = playlist.get_playlist_list()

In [ ]:
import sys
sys.getsizeof(all_playlists)

In [ ]:
def get_tracks(playlist):
    track_list = set([])
    for i in range(playlist['num_tracks']):
        track_list.add(playlist['tracks'][i]['track_uri'])
    return track_list

In [ ]:
with open("track_uri.txt", "w") as text_file:
    for i in range(len(playlist_list)):
        for item in get_tracks(playlist_list[i]):
            text_file.write("%s " % item)
        text_file.write("\n")


In [ ]:
track_df=pd.read_csv("GloVe/vectors.txt",header=None, sep=' ')
track_df = track_df.set_index(0)
track_df.columns

In [ ]:
track_df.shape

In [ ]:
track_df.head()

In [ ]:
def get_playlist_df(playlist):
    track_list = get_tracks(playlist)
    track_list = track_list & set(track_df.index)
    playlist_df = track_df.loc[list(track_list), :]
    return playlist_df

In [ ]:
playlist_df_list = [get_playlist_df(playlist) for index, playlist in enumerate(playlist_list)]

In [ ]:
playlist_df_list[0].as_matrix()

In [ ]:
training_playlist = playlist_df_list[:10000]

In [ ]:
concat_training_playlist = pd.concat(training_playlist)

In [ ]:
def make_query_mediod(investigated_playlist, index_to_remove):
    query = investigated_playlist.drop(investigated_playlist.index[index_to_remove])
    query_distance_matrix = sklearn.metrics.pairwise.cosine_distances(query)
    mediod = np.argmin(query_distance_matrix.sum(axis = 0))
    return mediod

In [ ]:
def make_mediod_features(mediod, investigated_playlist):
    mediod_features = investigated_playlist.iloc[mediod]
    return mediod_features

In [ ]:
from sklearn.neighbors import NearestNeighbors
def cal_knn(k, concat_training_playlist):
    nbrs = NearestNeighbors(n_neighbors=k).fit(concat_training_playlist)
    all_true_indices = []
    all_distances = []
    all_indices = []
    for i in range(len(training_playlist)):
        if len(training_playlist[i]) < 5:
            continue
        for j in range(len(training_playlist[i])):
            true_indices = j
            all_true_indices.append(true_indices)
            mediod = make_query_mediod(training_playlist[i], j)
            mediod_features = make_mediod_features(mediod, training_playlist[i])
            distances, indices = nbrs.kneighbors(mediod_features.as_matrix().reshape(1,50))
            all_distances.append(distances)
            all_indices.append(indices)
    return all_true_indices, all_distances, all_indices

In [ ]:
all_true_indices, all_distances, all_indices = cal_knn(100, concat_training_playlist)

In [ ]:
# import pdb
# pdb.pm()

In [ ]:
def cal_ave_precision(k, all_true_indices, all_indices):
    sum_precision = 0.0
    for i in range(len(all_true_indices)):
        precision = float(all_true_indices[i] in all_indices[i][0, :k])
        sum_precision += precision
    average_precision = sum_precision / len(all_true_indices)
    return average_precision

In [ ]:
for k in [1,5,10,20]:
    print(k, 100* cal_ave_precision(k, all_true_indices, all_indices))